## Import libraries

In [36]:
import numpy as np

#l5kit imports  
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from l5kit.data import PERCEPTION_LABELS
import os

import torch
import pandas as pd

## Configuration

In [2]:
cfg = {
    'format_version': 4,
    'data_path': "../input/lyft-motion-prediction-autonomous-vehicles/",
    'model_params': {
        'model_architecture': 'resnet18',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'sample_data_loader': {
        'key': 'scenes/sample.zarr',
        'batch_size': 16,
        'shuffle': False,
        'num_workers': 4
    },
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    
    'val_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 16,
        'shuffle': False,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'train_params': {
        'max_num_steps': 12000,
        'checkpoint_every_n_steps': 500,
    }
}

## Loading CUDA device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device {device}')

device cuda:0


## Loading Dataset

In [4]:
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
rasterizer = build_rasterizer(cfg, dm)

In [35]:
"""
The agents['label_probabilities'] cannot be loaded entirely into kaggle memory
Hence dividing the dataset into batches and summing up at the end

Using GPU to run argmax improves runtime massively
"""

def count_Agentlabels_zarr(data_loader_key):
    dataset_path = dm.require(cfg[data_loader_key]["key"])
    zarr_dataset = ChunkedDataset(dataset_path).open()
    
    # get agents from dataset
    agents = zarr_dataset.agents
    no_batches = 8
    batch_len = round(len(agents) / no_batches )
    total_count = []

    for i in range(no_batches):
        probabilities = torch.from_numpy(agents[i * batch_len : (i+1) * batch_len]["label_probabilities"]).to(device)
        labels_indexes = torch.argmax(probabilities, dim=1)

        counts = []
        for idx_label, label in enumerate(PERCEPTION_LABELS):
            counts.append(torch.sum(labels_indexes == idx_label))
        counts = [x.cpu().numpy().item() for x in counts]
        print(f'{i} batch result = {counts}')
        total_count.append(counts)
        del(probabilities)

    agent_count =  np.sum(total_count, axis = 0).tolist()
    print(f'Agents distribution for {data_loader_key} is ', agent_count)

In [11]:
%%time

""" Uncomment following lines to run the code """
#train_counts = count_Agentlabels_zarr("train_data_loader")
#val_counts = count_Agentlabels_zarr("val_data_loader")
#test_counts = count_Agentlabels_zarr("test_data_loader")

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs


## Results for Agent distribution in differnt datasets

In [26]:
train_counts = [0, 214185074, 0, 96737892, 0, 0, 0, 0, 0, 0, 0, 0, 1667331, 0, 7534327, 0, 0]
val_counts   = [0, 209417891, 0, 94918785, 0, 0, 0, 0, 0, 0, 0, 0, 1574639, 0, 6706572, 0, 0]
test_counts  = [0, 58143776, 0, 27888998, 0, 0, 0, 0, 0, 0, 0, 0, 486857, 0, 2075290, 0, 0]

In [31]:
df = pd.DataFrame({"LABELS" : PERCEPTION_LABELS, 
                   "train_counts" : train_counts, 
                   "val_counts" : val_counts,
                   "test_counts" : test_counts
                  })

df = df.set_index('LABELS')

# selecting only necessary rows
df = df[(df.T != 0).any()]
df = df.drop('PERCEPTION_LABEL_UNKNOWN')

In [32]:
df

,train_counts,val_counts,test_counts
LABELS,,,
PERCEPTION_LABEL_CAR,96737892,94918785,27888998
PERCEPTION_LABEL_CYCLIST,1667331,1574639,486857
PERCEPTION_LABEL_PEDESTRIAN,7534327,6706572,2075290


In [33]:
# normalising to percentages
for dataset in ['train_counts', 'val_counts', 'test_counts']:
    df[dataset] = df[dataset] / df[dataset].sum()

In [34]:
df

,train_counts,val_counts,test_counts
LABELS,,,
PERCEPTION_LABEL_CAR,0.913142,0.919756,0.915860
PERCEPTION_LABEL_CYCLIST,0.015739,0.015258,0.015988
PERCEPTION_LABEL_PEDESTRIAN,0.071119,0.064986,0.068151


## Conclusion

> ### 1. Distribution of different agents is equal for all the three datasets, which is very crucial for consistent results across training, validation and test sets. 
  
> ### 2. Percentages of Pedestrian is around 7%, so increasing the pixel resolution might have will most likely have impact in max 7% increase (decisive for top kagglers, less for entry-level people). So, decision on pixel resolution can be decided based on tradeoff between resource and performance objective


